## Electron ptychography cheat sheet

A ptychography experiment should satisfy sampling and overlap criteria:

### Sampling

Things go much smoother if the diffraction pattern is fully sampled on the detector. The real space pixel size is given by 
$$\Delta x = \lambda / \theta_D,$$
where 
$$\theta_D = {D \over L}$$
is the lateral opening angle of the detector, with $D = N s = 256 \times 55 \mathrm{\mu m} = 14.08 \mathrm{mm}$ is the width of the Merlin detector and $L$ the camera length.

In our case the Nyquist criterion is equivalent to the statement that the illumination must not be wider than half the real-space field of view:
$$w < \frac{N}{2} \Delta x,$$
or
$$w < \frac{\lambda L}{2 s}.$$
If $\alpha$ is the convergence semi-angle and the sample is defocused by a distance $d$, then $w \approx 2 \alpha d$, and 
$$d < \frac{\lambda L}{4 \alpha s}.$$
For $30\;\mathrm{keV}$ electrons, $L = 18.3\;\mathrm{cm}$ and $\alpha = 24.2\;\mathrm{mrad}$, the defocus should never be larger than $240\;\mathrm{nm}$. For now I'd suggest that we not venture further than half of this limit.


### Overlap

To ensure sufficient redundancy in the dataset, neighbouring illuminated regions have to overlap. A possible rule of thumb is to adjust the step size so that, for the inner part of the scanned area, every part is illuminated more than once. With a disc illumination on a square lattice, this means a step size roughly $35\%$ the diameter of the probe. From what we obtained above, we get that the step size $l$ (sorry I'm running out of variables already) is limited by the inequality
$$l < 0.35 w$$

### Reciprocal space overlap

This is relevant only when imaging periodic samples. Information on the position the probe within a unit cell is encoded in the interference between the Bragg discs. To ensure that this is not a problem, the aperture size has to be large enough. For a unit cell width $a$, we must have
$$\alpha > \frac{\lambda}{2a}.$$


In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

In [2]:
# Constants
detector_pixel_size = 220e-6
N = 64  # Number of lateral pixels on the merlin detector 
electron_rest_mass_eV = 510998.9461
hc_m_eV = 1.23984197e-6 

In [3]:
# Initial parameters
electron_energy_eV = 80000.
semi_angle = .020

wavelength = hc_m_eV / np.sqrt(electron_energy_eV*(electron_energy_eV + 2*electron_rest_mass_eV))

In [4]:
# Suggested procedure: first choose a target pixel size
# Example here: 0.5 Angstrom
dx = .4076e-10

### 1) Compute camera length
camera_length = dx * N * detector_pixel_size / wavelength

# Apparently we don't know the exact camera length
# so instead we can compute the expected pupil radius on the detector
target_pupil_radius = camera_length * semi_angle

# Now we would change the camera length until the pupil radius target is reached

### 2) Select defocus

# Given the pixel size we can accommodate the width of an illumination of at most dx * N / 2 
# This informs us on the maximum defocus (additional 1/2 because we divide by semi-angle)
max_defocus = .25*dx*N / semi_angle

# Let's say we chose half this value 
defocus = .5 * max_defocus

# The illumination width will be 
probe_width = 2*semi_angle*defocus

# And the maximum step size
max_step_size = .35 * probe_width

# Again, it is ok to be conservative
step_size = .5*max_step_size

# ... and it is nicer to refine it to be an integral number of pixels
refined_step_size = dx * np.floor(step_size/dx)

In [5]:
print('Camera length: %f mm' % (camera_length*1e3))
print('Defocus: %f nm' % (defocus*1e9))
print('Scanning step size: %f nm' % (refined_step_size*1e9))

Camera length: 137.437698 mm
Defocus: 16.304000 nm
Scanning step size: 0.081520 nm


In [8]:
refined_step_size

8.1520000000000006e-11

#### Code used to estimate the pupil radius

In [ ]:
from scipy import ndimage as ndi

# Assuming data is a (nx, ny, 256, 256) array.
# Compute frame average
dm = data.mean(axis=0).mean(axis=0)

# Estimate pupil area through thresholding
pupil_mask = ndi.binary_closing(dm>.05*dm.max(), iterations=2)

# Radius based on area (in pixel units)
pupil_r = np.sqrt(pupil_mask.sum()/np.pi)

#### STEM analysis

In [ ]:
x0 = np.arange(data.shape[-1]).reshape((1,1,-1)) - data.shape[-1]/2.
s0 = data.sum(axis=-2)
s1 = data.sum(axis=-1)
trans = s0.sum(axis=-1)
dpcx = (s0*x0).sum(axis=-1)/trans
dpcy = (s1*x0).sum(axis=-1)/trans
plt.subplot(131); plt.imshow(trans); plt.title('Transmission');
plt.subplot(132); plt.imshow(dpcx); plt.title('Differential phase contrast x');
plt.subplot(133); plt.imshow(dpcy); plt.title('Differential phase contrast y');